# Visualizando los resultados

## Importar librerias **`xarray`**

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

## Leer datos

In [ ]:
ds = xr.open_dataset('./output/Bell_SMB_out_SEB.nc')
ds

In [ ]:
ds.acc.max()

## Graficando las series temporales

In [ ]:
ds.acc.mean(('lat', 'lon')).plot()

In [ ]:
ds.abl[:,:,:].mean(('lat', 'lon')).plot()

In [ ]:
ds.smb[:,:,:].mean(('lat', 'lon')).plot()

## Graficando espacialmente

In [ ]:
ds.sel(time='2008-07-05T06').acc[:,:].plot()

In [ ]:
ds.smb[0,:,:].plot()

## Crear una función para calcular el SMB anual

In [ ]:
def sum_annual_ds_2d(ds):
    year = np.unique(ds.time.dt.year.values)
    V_interp = []
    for i in range(len(year)-1):
        V_interp.append(ds.sel(time=slice(f'{year[i]}-04-01',f'{year[i]+1}-03-31')).sum('time').values)

    da_a = xr.DataArray(
        V_interp,
        dims=('time', 'lat', 'lon'),
        coords=[
            ('time', year[1:]),
            ('lat', ds.lat.values),
            ('lon', ds.lon.values),
        ],
    )
    da_a.lon.attrs['long_name'] = 'lon'
    da_a.lat.attrs['long_name'] = 'lat'

    return da_a


In [ ]:
SMB_yr_h = sum_annual_ds_2d(ds.smb).where(ds.MASK==1)

## Graficando el SMB para varios años

In [ ]:
print(SMB_yr_h.min())
print(SMB_yr_h.max())

SMB_yr_h.plot.imshow(x="lon", y="lat", col="time", col_wrap=3, robust=True, cmap='RdBu')


In [ ]:
SMB_yr_h.mean(('lat', 'lon')).to_dataframe(name="SMB").plot(kind='bar', figsize=(8,4))

## Crear una barra de colores personalizado

In [ ]:
import matplotlib.colors as colors

In [ ]:
negatives = -3.2
positives =  0.1

bounds_min = np.linspace(negatives, 0, 129)
bounds_max = np.linspace(0, positives, 129)[1:]
    # the zero is only needed once
    # in total there will be 257 bounds, so 256 bins
bounds = np.concatenate((bounds_min, bounds_max), axis=None)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)

num_neg_colors = int(256 / (positives - negatives) * (-negatives))
num_pos_colors = 256 - num_neg_colors
cmap_BuRd = plt.cm.RdBu
colors_2neg_4pos = [cmap_BuRd(0.5*c/num_neg_colors) for c in range(num_neg_colors)] + [cmap_BuRd(1-0.5*c/num_pos_colors) for c in range(num_pos_colors)][::-1]
cmap_2neg_4pos = colors.LinearSegmentedColormap.from_list('cmap_2neg_4pos', colors_2neg_4pos, N=256)


In [ ]:
SMB_yr_h.plot.imshow(x="lon", y="lat", col="time", col_wrap=3, robust=True, cmap=cmap_2neg_4pos, vmin=-3.4, vmax=0.1)